# 125个经典技术指标择时分析
## 广发证券 2020年1月3日

##### TODO: 
1. 增加新因子
2. 将因子作为数据导入

In [1]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import os

# 回测框架
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeed
import indicatorJ as indJ

# 基础函数
import utilsJ

## 策略主体

### 新因子测试

In [2]:
class gf125_test(bt.Strategy):
    
    params = (
        # Global params
        ('printlog', False),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        if self.p.printlog or doprint:
            dt = dt or self.data.datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        
        # Initialization
        self.buyprice = None
        self.sellprice = None
        
        # Indicators
        self.PAC = indJ.PAC(self.data)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                         (order.executed.price, order.executed.size,
                          self.broker.get_cash(), self.broker.get_value()))
                self.buyprice = order.executed.price

            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                        (order.executed.price, -order.executed.size,
                         self.broker.get_cash(), self.broker.get_value()))
                self.sellprice = order.executed.price

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):
        # if self.order:
        #     return
        print(self.data.datetime.date(0))
        print(self.data.high[-1])
        print(self.data.high[0])
        print(self.PAC.lines.Upper[0])

### 策略主体

In [4]:
class gf125(bt.Strategy):
    
    params = (
        ('printlog', False),
        ('N1', 1), 
        ('N2', 5),
        ('N3', 10),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.data.datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        
        # Initialization
        self.buyprice = None
        self.sellprice = None
        self.order = None
        
        # Indicators
        ## 价格动量类因子
        self.ER = indJ.ER(self.data, plot=False)
        self.DPO = indJ.DPO(self.data, plot=False)
        self.POS = indJ.POS(self.data, plot=False)
        self.TII = indJ.TII(self.data, plot=False)
        self.ADTM = indJ.ADTM(self.data, plot=False)
        self.MADisplaced = indJ.MADisplaced(self.data, plot=False)
        self.T3 = indJ.T3(self.data, plot=False)
        self.VMA = indJ.VMA(self.data, plot=False)
        self.BIAS = indJ.BIAS(self.data, plot=False)
        self.TMA = indJ.TMA(self.data, plot=False)
        self.TYP = indJ.TYP(self.data, plot=False)
        self.WMA_BS = indJ.WMA_BS(self.data, plot=False)
        self.PAC = indJ.PAC(self.data, plot=False)
        self.MTM = indJ.MTM(self.data, plot=False)

        ## 价格翻转类
        self.KDJ = indJ.KDJ(self.data, plot=False)
        self.RMI = indJ.RMI(self.data, plot=False)
        self.SKDJ = indJ.SKDJ(self.data, plot=False)
        self.CCI = indJ.CCI(self.data, plot=False)

        ## 成交量类
        self.MAAMT = indJ.MAAMT(self.data, plot=False)
        self.SROCVOL = indJ.SROCVOL(self.data, plot=False)

        # Add trend indicator
        self.uptrend = bt.And(self.data.low == btind.Highest(self.data.low, period=self.p.N3), 
                              self.data.high == btind.Highest(self.data.high, period=self.p.N3))
        

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                         (order.executed.price, order.executed.size,
                          self.broker.get_cash(), self.broker.get_value()))
                self.buyprice = order.executed.price

            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                        (order.executed.price, -order.executed.size,
                         self.broker.get_cash(), self.broker.get_value()))
                self.sellprice = order.executed.price

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):
        if self.order:
           return

        # Major votes
        class1_buy = [self.ER.lines.Buy[0], self.DPO.lines.Buy[0], self.POS.lines.Buy[0], self.TII.lines.Buy[0],
                      self.ADTM.lines.Buy[0], self.MADisplaced.lines.Buy[0], self.T3.lines.Buy[0], self.VMA.lines.Buy[0],
                      self.BIAS.lines.Buy[0], self.TMA.lines.Buy[0], self.TYP.lines.Buy[0], self.WMA_BS.lines.Buy[0],
                      self.PAC.lines.Buy[0], self.MTM.lines.Buy[0]]
        class2_buy = [self.KDJ.lines.Buy[0], self.RMI.lines.Buy[0], self.SKDJ.lines.Buy[0], self.CCI.lines.Buy[0]]
        class3_buy = [self.MAAMT.lines.Buy[0], self.SROCVOL.lines.Buy[0]]

        class1_sell = [self.ER.lines.Sell[0], self.DPO.lines.Sell[0], self.POS.lines.Sell[0], self.TII.lines.Sell[0],
                       self.ADTM.lines.Sell[0], self.MADisplaced.lines.Sell[0], self.T3.lines.Sell[0], self.VMA.lines.Sell[0],
                       self.BIAS.lines.Sell[0], self.TMA.lines.Sell[0], self.TYP.lines.Sell[0], self.WMA_BS.lines.Sell[0],
                       self.PAC.lines.Sell[0], self.MTM.lines.Sell[0]]
        class2_sell = [self.KDJ.lines.Sell[0], self.RMI.lines.Sell[0], self.SKDJ.lines.Sell[0], self.CCI.lines.Sell[0]]
        class3_sell = [self.MAAMT.lines.Sell[0], self.SROCVOL.lines.Sell[0]]

        major_vote_buy = max(class1_buy, key=class1_buy.count) + max(class2_buy, key=class2_buy.count) + max(class3_buy, key=class3_buy.count)
        major_vote_sell = max(class1_sell, key=class1_sell.count) + max(class2_sell, key=class2_sell.count) + max(class3_sell, key=class3_sell.count)
        

        if major_vote_buy > 0 and self.getposition(self.data).size == 0 and self.uptrend[0]:
            lots = np.floor(self.broker.getcash() / (100*self.data.close[0]))*100
            self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % (self.data.close[0], lots, self.getposition(self.data).size))
            self.order = self.buy(size=lots)
        elif major_vote_sell > 0 and self.getposition(self.data).size != 0 and self.uptrend[0]:
            self.log('Sell CREATE (Close), Price: %.2f, Current Position: %i' % (self.data.close[0], self.getposition(self.data).size))
            self.order = self.close()

## 回测

### 新因子测试

In [ ]:
startdate = dt.datetime(2019,12,31)
enddate = dt.datetime(2022,1,1)
stock_code = '600588.SH'


if __name__ ==  '__main__':
    
    # Initialization
    cerebro = bt.Cerebro()
    strats = cerebro.addstrategy(gf125_test, printlog=True)

    # Data
    df = utilsJ.get_stock(stock_code, startdate, enddate)
    data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)
    cerebro.adddata(data)

    # Start condition
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(200000)
    #cerebro.broker.setcommission()
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Execution
    cerebro.run()

    # Final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))

    # Visualization
    #cerebro.plot(iplot=False)

### 单股回测

In [ ]:
startdate = dt.date(2019,6,30)
enddate = dt.date(2022,10,28)
stock_code = '000005.SZ'


if __name__ ==  '__main__':
    
    # Initialization
    cerebro = bt.Cerebro()
    strats = cerebro.addstrategy(gf125, printlog=True)

    # Data
    df = utilsJ.get_stock(stock_code, startdate, enddate)
    data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)
    cerebro.adddata(data)

    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(200000)
    #cerebro.broker.setcommission()
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Execution
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))

    # Visualization
    cerebro.plot(iplot=False)

### 沪深300/全市场回测

In [ ]:
startdate = dt.datetime(2019,12,31)
enddate = dt.datetime(2022,10,28)
index_code = '000300.SH'
profit_dict = {}

if __name__ ==  '__main__':
    
    index_list = utilsJ.get_index_components(index_code, startdate, enddate)
    
    for stk in index_list:
        
        try:
            # Initialization
            cerebro = bt.Cerebro()
            strats = cerebro.addstrategy(gf125, printlog=True)

            # Data
            df = utilsJ.get_stock(stk, startdate, enddate)
            data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)
            cerebro.adddata(data)

            # Start condition
            cerebro.broker = bt.brokers.BackBroker(coc=True)   
            cerebro.broker.setcash(2000000)
            #cerebro.broker.setcommission()
            start_value = cerebro.broker.getvalue()
            #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
            cerebro.run()
            final_value = cerebro.broker.getvalue()
            #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
            print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))
            profit_dict[stock_code] = (final_value - start_value) / start_value * 100
        
        except:
            print('Execution failed. Code: ' + stk)